# Getting and Cleaning Data - Week 1
Note: Copyright preserved by Prof. Jeffrey Leek at Johns Hopkins Bloomberg School of Public Health, and "Getting and Cleaning Data" course in "Data Science" specialization at <www.coursera.org>

The goal of this course: **Raw data -> Processing script -> tidy data** -> data analysis -> data communication

## Raw and Processed Data

### Definition of data
+ Data are values of **qualitative** or **quantitative** variables, belonging to a **set of items**.
+ **Variables**: A measurement or characteristic of an item.

### Raw versus processed data

#### Raw data

+ The original source of the data
+ **Often hard to use for data analyses**
+ Data analysis *includes* processing
+ Raw data may only need to be processed once

#### Processed data

+ Data that is ready for analysis
+ Processing can include merging, subsetting, transforming, etc.
+ There may be standards for processing
+ **All steps should be recorded**

## The components of tidy data

### The four things you should have

1. The raw data
2. A tidy data set
3. A code book describing each variable and its values in the tidy data set.
4. **An explicit and exact recipe you used to go from 1 -> 2, 3.**

### The raw data

*You know the raw data is the right format if you*
1. **Ran no software on the data**
2. Did not manipulate any of the numbers in the data
3. You did not remove any data form the data set
4. You did not summarize the data in any way

### The tidy data

1. **Each variable you measure should be in one column**
2. Each different observation of that variable should be in a different row
3. There should be one table for each "kind" of variable
4. If you have multiple tables, they should include a column in the table that allows them to be linked

*Some other important tips*
+ Include a row at the top of each file with variable names
+ Make variable names human readable, eg., AgeAtDiagnosis instead of AgeDx
+ In general data should be saved in one file per table

### The code book

1. Information about the variables (including units!) in the data set not contained in the tidy data
2. Information about the summary choices you made
3. Information about the experimental study design you used

*Some other important tips*
+ A common format for this document is a Word/text file
+ There should be a section called "Study design" that has a thorough description of how you collected the data
+ There must be a section called "Code book" that describes each variable and its units.

### The instruction list

+ Ideally a computer script
+ The input for the script is the raw data
+ The output is the processed, tidy data
+ There are no parameters to the script

In some cases it will not be possible to script every step. In that case you should provide instructions like:
1. Step 1 - take the raw file, run version 3..2 of summarize software with parameters a = 1, b = 2, c = 3
2. Step 2 - run the software separately for each sample
3. Step 3 - take column three of outputfile.out for each sample and that is the corresponding row in the output data set

## Downloading files

### Get/set your working directory

+ A basic component of working with data is knowing your working directory
+ The two main commands are `getwd()` and `setwd()`
+ Be aware of relative versus absolute paths
  - **Relative** - `setwd("./data")`, `setwd("../")`
  - **Absolute** - `setwd("/Users/jtleek/data/")`
+ Important difference in Windows - `Setwd("C:\\Users\\Andrew\\Downloads")`

### Checking for and creating directories

+ `file.exists("directoryName")` will check to see if the directory exists
+ `dir.create("directoryName")` will create a directory if it doesn't exist
+ An example on checking for a "data" directory and creating it if it doesn't exist
```{r}
if (!file.exists("data")) {
    dir.create("data")
}
```

### Getting data from the internet - `download.file()`

+ Downloads a file from the internet
+ Even if you could do this by hand, helps with reproducibility
+ Important parameters are *url, destfile, method*
+ Useful for downloading tab-delimited, csv, and other files
+ Example
```{r}
fileUrl <- "https://data.baltimorecity.gov/api/views/dz54-2aru/rows.csv?accessType=DOWNLOAD"
download.file(fileUrl, destfile = "./data/cameras.csv", method = "curl")
list.files("./data")
## [1] "cameras.csv"
```
```{r}
dateDownloaded <- date()
dateDownloaded
## [1] "Sun Jan 12 21:37:44 2014"
```

*Some notes about `download.file()`*
+ If the url starts with *http* you can use `download.file()`
+ If the url starts with *https* on Windows you may be ok
+ If the url starts with *https* on Mac you may need to set `method = "curl"`
+ If the file is big, this might take a while
+ Be sure to record when you downloaded

## Reading local flat files

### Loading flat files - `read.table()`

+ This is the main function for reading data into R
+ Flexible and robust but requires more parameters
+ Reads the data into RAM - big data can cause problems
+ Important parameters *file, header, sep, row.names, nrows*
+ Related: `read.csv()`, `read.csv2()`
+ Example
```{r}
# Download the file to load
if (!file.exists("data")) {
    dir.create("data")
}
fileUrl <- "https://data.baltimorecity.gov/api/views/dz54-2aru/rows.csv?accessType=DOWNLOAD"
download.file(fileUrl, destfile = "./data/cameras.csv", method = "curl")
dateDownloaded <- date()
```
```{r}
cameraData <- read.table("./data/camers.csv")
## Error: line 1 did not have 13 elements
```
```{r}
cameraData <- read.table("./data/cameras.csv", sep = ",", header = TRUE)
```

### Loading flat files - `read.csv()`

+ `read.csv()` sets *sep = ","* and *header = TRUE*
```{r}
cameraData <- read.csv("./data/camera.csv")
```

### Some more important parameters

+ `quote` - you can tell R whether there are any quoted values, `quote = ""` means no quotes
+ `na.strings` - set the character that represents a missing value
+ `nrows` - how many rows to read of the file (e.g. `nrows = 10` reads 10 lines)
+ `skip` - number of lines to skip before starting to read
+ Experience: the biggest trouble with reading flat files are quotation marks **`** or **"** placed in data values, setting `quote = ""` often resolves these

## Reading Excel files

### `read.xlsx()`, `read.xlsx2()` `{xlsx package}`

```{r}
# Download the file to load
if (!file.exists("data")) {
    dir.create("data")
}
fileUrl <- "https://data.baltimorecity.gov/api/views/dz54-2aru/rows.xlsx?accessType=DOWNLOAD"
download.file(fileUrl, destfile = "./data/cameras.xlsx", method = "curl")
dateDownloaded <- date()
```
```{r}
library(xlsx)
cameraData <- read.xlsx("./data/cameras.xlsx", sheetIndex = 1, header = TRUE)
```
```{r}
# Reading specific rows and columns
cameraData <- read.xlsx("./data/cameras.xlsx", sheetIndex = 1,
                        colIndex = 2:3, rowIndex = 1:4)
```

*Further notes*
+ The `write.xlsx` function will write out an Excel file with similiar arguments
+ `read.xlsx2` is much faster then `read.xlsx` but for reading subsets of rows may be slightly unstable
+ The `XLConnect`package has more options for writing and manipulating Excel files
+ The **XLConnect vignette** is a good place to start for that package
+ In general it is advised to store your data in either a database or in comma separated files (.csv) or tab separated files (.tab/.txt) as they are easier to distribute

## Reading XML

### XML

+ Extensive markup language
+ Frequently used to store structured data
+ Particularly widely used in internet applications
+ Extracting XML is the basis for most web scraping
+ Components
  - Markup - labels that give the text structure
  - Content - the actual text of the document

### Tags, elements, and attributes

+ Tags correspond to general labels
  - Start tags *\<section>*
  - End tags *\</section>*
  - Empty tags *\<line-break />*
+ Elements are specific examples of tags
  - *\<Greeting> Hello, world \</Greeting>*
+ Attrbutes are components of the label
  - *\<img src="jeff.jpg" alt="instructor"/>*
  - *\<step number="3"> Connect A to B. \</step>*

### Read the file into R

```{r}
library(XML)
fileUrl <- "http://www.w3schools.com/xml/simple.xml"
doc <- xmlTreeParse(fileUrl, useInternal = TRUE)
rootNode <- xmlRoot(doc)
xmlName(rootNode)
## [1] "breakfast_menu"
names(rootNode)
##   food   food   food   food   food
## "food" "food" "food" "food" "food"
```

### Directly access parts of the XML document

```{r}
rootNode[[1]]
## <food>
##   <name>Belgian Waffles</name>
##   <price>$5.95</price>
##   <description>Two of our famous Belgian Waffles with plenty of real maple syrup</description>
##   <calories>650</calories>
## </food>
```
```{r}
rootNode[[1]][[1]]
## <name>Belgian Waffles</name>
```

### Programatically extract parts of the file
```{r}
xmlSApply(rootNode, xmlValue)
```

### XPath
+ */node* Top level node
+ *//node* Node at any level
+ *node[@attr-name]* Node with an attribute name
+ *node[@attr-name='bob']* Node with attribute name attr-name='bob'
+ Information from <https://www.stat.berkeley.edu/~statcur/Workshop2/Presentations/XML.pdf>
```{r}
# Get the items on the menu and prices
xpathSApply(rootNode, "//name", xmlValue)
## [1] "Belgian Waffles"        "Strawberry Belgian Waffles"        "Berry-Berry Belgian Waffles"
## [4] "French Toast"           "Homestyle Breakfast"
xpathSApply(rootNode, "//price", xmlValue)
## [1] "$5.95" "$7.95" "$8.95" "$4.50" "$6.95"
```
```{r}
# Another example - Extract content by attributes
fileUrl <- "http://espn.go.com/nfl/team/_/name/bal/baltimore-ravens"
doc <- htmlTreeParse(fileUrl, useInternal = TRUE)
scores <- xpathSApply(doc, "//li[@class='score']", xmlValue)
teams <- xpathSApply(doc, "//li[@class='team-name']", xmlValue)
scores
## [1] "49-27"    "14-6"     "30-9" "23-20" "26-23" "19-17" "19-16" "24-18"
## [9] "20-17 OT" "23-20 OT" "19-3" "22-20" "29-26" "18-16" "41-7"  "34-17"
teams
##  [1] "Denver"     "Cleveland" "Houston"     "Buffalo"    "Miami"    "Green Bay"
##  [7] "Pittsburgh" "Cleveland" "Cincinnati"  "Chicago"    "New York" "Pittsburgh"
## [13] "Minnesota"  "Detroit"   "New England" "Cincinnati"
```